In [1]:
import numpy
import netCDF4
from osgeo import gdal, osr
#from datetime import datetime, date
import datetime
import os

#siloDIR = 'P:/projects/SILO/Wes/mwet/'
siloDIR = r'C:\DDrive\Maria\Lachie_netCDF'
#baseOutputDIR = 'C:/DDrive/SILO/tifs/mwet'
#baseOutputDIR = 'C:/DDrive/SILO/tifs/mwetTest'
#baseOutputDIR = 'C:\DDrive\Maria\TIFs'

baseOutputDIR = r'C:\DDrive\Maria\Lachie_netCDF\TIFS'


#baseOutputDIR = 'P:/projects/SILO/mwet44_posHeight'
#baseOutputDIR = 'P:/projects/SILO/Wes/tifs/mwet'
#baseOutputDIR = 'P:/projects/SILO/mwet10_negHeightRevArray'
dataVarName = 'daily_rain'
#dataVarName = 'et_morton_wet'
#fileNameEndBit = '_mwet.tif'
fileNameEndBit = '_rain.tif'

#dataVars = {'daily_rain':'_rain.tif','et_morton_wet':'_mwet.tif'}

# startYear = 1980
# endYear = 1981

startYear = 2015
endYear = 2015



In [2]:
if not os.path.exists(baseOutputDIR):
    os.makedirs(baseOutputDIR)
    print("Made dir: " + outPath)


In [3]:
if not os.path.exists(baseOutputDIR + '\\' + dataVarName):
    os.makedirs(baseOutputDIR + '\\' + dataVarName)
    print("Made dir: " + baseOutputDIR + '\\' + dataVarName)

In [4]:
# for thisVar in dataVars:
#     if not os.path.exists(baseOutputDIR + '\\' + thisVar):
#         os.makedirs(baseOutputDIR + '\\' + thisVar)
#         print("Made dir: " + baseOutputDIR + '\\' + thisVar)

In [5]:
"""
startYear = 2018
endYear = 2019

siloDIR = 'P:/projects/SILO/Wes/mwet/'
#baseOutputDIR = 'C:/DDrive/SILO/tifs/mwet'
baseOutputDIR = 'C:/DDrive/SILO/tifs/mwetTest'
dataVarName = 'et_morton_wet'
fileNameEndBit = '_mwet.tif'
"""

# Loop over years
#for year in range(1961, 1990):
# range uses start and STOP values, so last value isn't processed
#for year in range(2000, 2002):
for year in range(startYear, endYear + 1):
    print('Current year: ' + str(year))
    
    #create necessary dir for this year
    thisYearDir = baseOutputDIR + '/' + dataVarName + '/' + str(year)
    if not os.path.exists(thisYearDir):
        os.makedirs(thisYearDir)
    
    theFile = siloDIR + '\{:d}.'.format(year) + dataVarName + '.nc'
    #with netCDF4.Dataset('{:d}.monthly_rain.nc'.format(year), 'r') as dataset:
    print('Processing: ' + theFile)
    #with netCDF4.Dataset(siloDIR + '{:d}.'.format(year) + dataVarName + '.nc', 'r') as theDataSet:
    
    theDataSet = gdal.Open(theFile)
    
    #Recent GDAL Upgrade means that the GT object is getting read OK
    #Perhaps don't need the netCDF pixel version stuff below...
    theSiloGT = theDataSet.GetGeoTransform()
    #print("SiloGT: ")
    print(theSiloGT)
    
    """
    #print( "Driver: ",theDataSet.GetDriver().ShortName, theDataSet.GetDriver().LongName)
    #print( "Size is ", theDataSet.RasterXSize, theDataSet.RasterYSize)
    #print( "Bands = ", theDataSet.RasterCount)
    #print( "Coordinate System is:", theDataSet.GetProjectionRef ())
    #print( "GetGeoTransform() = ", theDataSet.GetGeoTransform ())
        
    #Need some vals for creating output extent etc
    #The GeoTransform object is:
    
    #GeoTransform[0] /* top left x */
    #GeoTransform[1] /* w-e pixel resolution */
    #GeoTransform[2] /* 0 */
    #GeoTransform[3] /* top left y */
    #GeoTransform[4] /* 0 */
    #GeoTransform[5] /* n-s pixel resolution (negative value) */
    
    #zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
    #y_max is the SOUTHERN value! For us, that's MIN! Oh well...
    
    netCDFData = netCDF4.Dataset(theFile, 'r')
    #print(netCDFData)
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    #These need to be adjusted so that we get pixel edges shifted, leaving
    #our SILO coords mid-pixel
    pixWidth = 0.05
    
    #Source Gdal loading looks at this pixHeight value to work out the lower left coordinate
    #Either way, Source will interpret inGeoTransform[3] as being our North coordinate,
    #calculating the yllCorner to be a MORE NEGATIVE value.
    
    #xllCorner = inGeoTransform[0];
    #yllCorner = (inGeoTransform[5] > 0)?inGeoTransform[3] - ( inGeoTransform[5] * inDataset.RasterYSize ):
    #                                   inGeoTransform[3] + ( inGeoTransform[5] * inDataset.RasterYSize );
    
    pixHeight = pixWidth * -1
    #pixHeight = pixWidth
    
    #x_min = lons[:1] - (0.5 * pixWidth)
    x_min = min(lons) - (0.5 * pixWidth)
    
    #This gives first value, which in SILO is the smallest or 'most negative' or 'most southerly'
    #y_min = lats[:1] - (0.5 * pixHeight)
    
    #This gives last value, which in SILO is the largest or 'most positive' or 'most northerly'
    #y_min = lats[len(lats)-1:] - (0.5 * pixHeight)
    
    #-10
    y_min = max(lats) - (0.5 * pixHeight)
    
    #-44
    #yes, use lower left if supplying positive pixel height
    #y_min = min(lats) - (0.5 * pixHeight)
    
    
    netCDFData.close()
    """
    
    dStart = datetime.date(int(year), 1, 1)
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)#WGS84

    theDataSet.SetProjection(srs.ExportToWkt())
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    for band in range( theDataSet.RasterCount ):
           
        dateAdd = band
        dateNow = dStart + datetime.timedelta(days=dateAdd)
        
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        
        band += 1
        #print("[ GETTING BAND ]: ", band)
        srcband = theDataSet.GetRasterBand(band)
        
        if srcband is None:
            continue
        
        scaleFactor = 1.0
        
        #Recent GDAL Upgrade means that the dataVarName isn't needed here
        #sf = srcband.GetMetadataItem(dataVarName + '_scale_factor')
        #nd = srcband.GetMetadataItem(dataVarName + '__FillValue')
        sf = srcband.GetMetadataItem('scale_factor')
        nd = srcband.GetMetadataItem('_FillValue')
        off = srcband.GetMetadataItem('add_offset')
        
        #print(sf)
        
        scaleFactor = float(sf)
        rawNoData = float(nd)
        offset = float(off)
        if band == 1:
            print(offset)
            
        rasterOutputFile = thisYearDir + '/' + dateForFileName + fileNameEndBit
        outRaster = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, theDataSet.RasterXSize, theDataSet.RasterYSize, 1, gdal.GDT_Float32)
        #outRaster.SetGeoTransform((x_min, pixWidth, 0, y_min, 0, pixHeight))
        outRaster.SetGeoTransform(theSiloGT)
        outBand = outRaster.GetRasterBand(1)
        bandArray = srcband.ReadAsArray()
        
        #Why does gdal_grid turn image upside-down?
        #revert the array lines
        #A few things to note:
        #
        #The point used in the geotransform should be upper-left
        #The resolution in y-direction should be negative
        #In the projection (at least the ones I use) positive y-direction is up
        #In numpy arrays positive y-direction is down
        #When using gdal's WriteArray it uses the upper left corner
        #
        #Hope this helps other people's confusion.
        
        #Change in GDAL version means that WriteArray no longer needs an inversion of Y axis
        #bandArray = bandArray[::-1,:]
        outBand.WriteArray((bandArray * scaleFactor) + offset)
        outBand.SetNoDataValue((rawNoData * scaleFactor) + offset)
        outRaster.SetProjection(srs.ExportToWkt())
        outBand.FlushCache()
    
    

print("All Done")

Current year: 2015
Processing: C:\DDrive\Maria\Lachie_netCDF\2015.daily_rain.nc
(111.975, 0.05, 0.0, -9.975, 0.0, -0.05)
3276.5
All Done
